##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow 2 quickstart for beginners

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Build a neural network that classifies images.
2. Train this neural network.
3. And, finally, evaluate the accuracy of the model.

This is a [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) notebook file. Python programs are run directly in the browser—a great way to learn and use TensorFlow. To follow this tutorial, run the notebook in Google Colab by clicking the button at the top of this page.

1. In Colab, connect to a Python runtime: At the top-right of the menu bar, select *CONNECT*.
2. Run all the notebook code cells: Select *Runtime* > *Run all*.

Download and install TensorFlow 2. Import TensorFlow into your program:

Note: Upgrade `pip` to install the TensorFlow 2 package. See the [install guide](https://www.tensorflow.org/install) for details.

In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.6.0


Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the samples from integers to floating-point numbers:

載入並將影像數值從0-255變成0-1 (MinMaxScaling)
mnist.load_data() return Tuple of NumPy arrays

https://www.tensorflow.org/api_docs/python/tf/keras/datasets/mnist/load_data

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11501568/11490434 [==============================] - 0s 0us/step


In [3]:
print ("x_train shape:", x_train.shape)
print ("x_train examples:", x_train[:2])
print ("y_train shape:", y_train.shape)
print ("y_train examples:", y_train[:2])
print ("input types:")
print (type(x_train[:2]))
print (type(y_train[:2]))
print ("train size:",x_train.shape[0] )

x_train shape: (60000, 28, 28)
x_train examples: [[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
y_train shape: (60000,)
y_train examples: [5 0]
input types:
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
train size: 60000


Build the `tf.keras.Sequential` model by stacking layers. Choose an optimizer and loss function for training:

建模時要確定Shape相容

Dropout的實驗值0.2是個超參數可調整

In [4]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


因為在分類時，NN用 wx + b = y Dense+Sigmoid，Dense的輸出其實是logit,
Sigmoid 之後才是probability (p)
logit is Log Odds = Ln (p/1-p)

For each example the model returns a vector of "[logits](https://developers.google.com/machine-learning/glossary#logits)" or "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" scores, one for each class.

In [6]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.37454832,  0.6052903 ,  0.32408026, -0.46260408,  0.54713404,
        -0.06991155, -0.09328168, -0.2082741 , -0.84889805,  0.25979948]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to "probabilities" for each class: 

In [7]:
print (type(predictions))

<class 'numpy.ndarray'>


In [8]:
tf.nn.softmax(predictions)

<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[0.12749754, 0.1605875 , 0.12122265, 0.05519899, 0.15151471,
        0.08174766, 0.07985935, 0.07118446, 0.03751162, 0.11367553]],
      dtype=float32)>

In [9]:
tf.nn.softmax(predictions).numpy()

array([[0.12749754, 0.1605875 , 0.12122265, 0.05519899, 0.15151471,
        0.08174766, 0.07985935, 0.07118446, 0.03751162, 0.11367553]],
      dtype=float32)

Note: It is possible to bake this `tf.nn.softmax` in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to
provide an exact and numerically stable loss calculation for all models when using a softmax output. 

The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [10]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class:
It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [11]:
loss_fn(y_train[:1], predictions).numpy()

2.5041182

有哪些 metrics?

https://www.tensorflow.org/api_docs/python/tf/keras/metrics

In [12]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

The `Model.fit` method adjusts the model parameters to minimize the loss: 

In [13]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2918 - accuracy: 0.9155
Epoch 2/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.1409 - accuracy: 0.9580
Epoch 3/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.1049 - accuracy: 0.9682
Epoch 4/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0860 - accuracy: 0.9733
Epoch 5/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0728 - accuracy: 0.9777


The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" or "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

model.evaluate 的輸出是 [loss, accuracy]. from doc:

Returns the loss value & metrics values for the model in test mode.

In [14]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0770 - accuracy: 0.9744


[0.07704710960388184, 0.974399983882904]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [15]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [16]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[7.87269414e-08, 6.22251761e-08, 8.81447340e-05, 5.06922806e-05,
        6.45260026e-11, 2.51135415e-08, 5.37369628e-13, 9.99859095e-01,
        1.22136015e-07, 1.75025423e-06],
       [4.84684108e-07, 5.60671091e-04, 9.99434292e-01, 2.06735513e-06,
        7.28676187e-16, 1.07264526e-08, 4.79675712e-08, 3.54769958e-14,
        2.47714775e-06, 5.01007534e-11],
       [8.52900257e-07, 9.99579966e-01, 8.64165486e-05, 5.25949736e-06,
        2.37253989e-05, 1.47504625e-06, 2.59977060e-05, 1.66216632e-04,
        1.09495042e-04, 6.58729675e-07],
       [9.99826014e-01, 5.64601566e-09, 5.83880574e-05, 1.90399277e-07,
        4.93995422e-08, 1.12224225e-05, 2.68260083e-05, 5.58707579e-05,
        4.37956338e-08, 2.14433549e-05],
       [1.96093155e-04, 1.15949838e-07, 3.55943077e-04, 7.28635882e-07,
        7.72874117e-01, 4.61363754e-07, 1.27118915e-06, 1.04682858e-03,
        5.43318274e-05, 2.25470170e-01]], dtype=float32)>